In [1]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

In [2]:
from typing import Callable, Iterator

import jax
from jax import numpy as jnp

from datasets.mnist import load_mnist, MnistLoaderConfig
from rubicon.nns.convnet import ConvNetConfig, ConvNet

jax.devices()

Platform 'METAL' is experimental and not all JAX functionality may be correctly supported!


Metal device set to: Apple M2

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



W0000 00:00:1758492658.960717 1616866 mps_client.cc:510] WARNING: JAX Apple GPU support is experimental and not all JAX functionality is correctly supported!
I0000 00:00:1758492658.994572 1616866 service.cc:145] XLA service 0x14c009240 initialized for platform METAL (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1758492658.994615 1616866 service.cc:153]   StreamExecutor device (0): Metal, <undefined>
I0000 00:00:1758492658.997657 1616866 mps_client.cc:406] Using Simple allocator.
I0000 00:00:1758492658.997672 1616866 mps_client.cc:384] XLA backend will use up to 5726240768 bytes on device 0 for SimpleAllocator.


[METAL(id=0)]

In [3]:
def get_input_shape(iterator: Iterator[tuple[jnp.ndarray, jnp.ndarray]]) -> tuple[int, ...]:
    """Extract the input shape from the first batch of an iterator"""
    try:
        x_batch, _ = next(iter(iterator))
        return x_batch.shape[:]
    except StopIteration:
        raise ValueError("Empty iterator; cannot determine input shape")

def create_data_factory(config: MnistLoaderConfig) -> Callable[[], Iterator[tuple[jnp.ndarray, jnp.ndarray]]]:
    """Create a factory function for training and test iterators"""
    def data_factory():
        train_iter, test_iter = load_mnist(config=config)
        return train_iter, test_iter
    return data_factory


In [13]:
mnist_loader_config = MnistLoaderConfig(batch_size=32)
temp_train_iter, _ = load_mnist(mnist_loader_config)
input_shape = get_input_shape(temp_train_iter)

data_factory = create_data_factory(mnist_loader_config)

2025-09-22 01:20:15.116882: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2025-09-22 01:20:15.149791: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [14]:
config = ConvNetConfig(
    conv_filters=[32],
    kernel_sizes=[(3, 3)],
    dense_sizes=[10],
    batch_size=32,
    input_shape=input_shape,
    num_epochs=10,
)
net = ConvNet(config)
net.initialize(); net.initialized

True

In [15]:
training_history = net.train(
    data_factory=data_factory,
    return_metrics=True,
    verbose=True
)


2025-09-22 01:20:51.935736: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-09-22 01:20:54.398523: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 0: 
	Train loss: 0.7450
	Train accuracy: 0.8409
	Test loss: 0.3858
	Test accuracy: 0.9198


2025-09-22 01:21:25.925243: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-09-22 01:21:28.366006: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 1: 
	Train loss: 0.3444
	Train accuracy: 0.9307
	Test loss: 0.2972
	Test accuracy: 0.9444


2025-09-22 01:21:59.515625: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-09-22 01:22:01.937568: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


...

Epoch 7: 
	Train loss: 0.1709
	Train accuracy: 0.9759
	Test loss: 0.1665
	Test accuracy: 0.9752


2025-09-22 01:25:23.780245: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-09-22 01:25:26.176749: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 8: 
	Train loss: 0.1615
	Train accuracy: 0.9771
	Test loss: 0.1592
	Test accuracy: 0.9763


2025-09-22 01:25:57.780994: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-09-22 01:26:00.192408: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 9: 
	Train loss: 0.1536
	Train accuracy: 0.9792
	Test loss: 0.1533
	Test accuracy: 0.9779
